In [ ]:
!pip -q install google-genai

In [29]:
from IPython.display import HTML, Markdown

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [67]:
def agente_treinador(tec):

    treinador = Agent(
        name="agente_treinador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um treinador de pessoas desenvolvedoras, especializado em fornecer desafios práticos envolvendo a tecnologia
            ou tópico que o usuário informou. Você sempre estimula o aprendizado por meio de atividades cuja finalidade possa ser
            aplicada em cenários reais de desenvolvimento de [tipo de aplicação, ex: web, mobile, backend]. Baseado na tecnologia
            informada, gere 9 desafios práticos, apresentados em formato de lista numerada com uma breve descrição do problema e os
            critérios de sucesso esperados. Crie 3 desafios de nível iniciante (focados em conceitos fundamentais), 3 de nível
            intermediário (envolvendo a aplicação de múltiplos conceitos ou a resolução de problemas mais complexos) e 3 de nível
            avançado (que exigem conhecimento profundo da tecnologia e a capacidade de lidar com cenários desafiadores ou otimizações).
            Seja criativo nos desafios, buscando criar algo fora do convencional, como [exemplos de cenários não convencionais, mas
            relevantes para a tecnologia], mas que ainda assim não fuja do escopo de dificuldade pretendido para cada nível.
            Para cada nível, os desafios devem refletir a complexidade e os conhecimentos esperados de um profissional com essa
            experiência na tecnologia. Ao final de cada desafio, inclua uma breve sugestão de como o usuário pode verificar sua
            solução ou um conceito chave a ser aprendido com a atividade. Comunique-se de forma sucinta, objetiva e precisa.
        """,
        description="Agente que gera desafios",
        tools=[google_search]
    )

    entrada_agente_treinador = tec

    tentativa = call_agent(treinador, entrada_agente_treinador)
    return tentativa

In [81]:
tecnologia = input("Tecnologia escolhida: ")

desafios = agente_treinador(tecnologia)
display(Markdown(f"\n{desafios}"))

Tecnologia escolhida: Android



Aqui estão 9 desafios práticos de desenvolvimento Android, divididos em níveis iniciante, intermediário e avançado, projetados para aprimorar suas habilidades e conhecimentos na plataforma:

**Iniciante:**

1.  **Criar um Conversor de Unidades:** Desenvolva um aplicativo que converta medidas de temperatura (Celsius para Fahrenheit e vice-versa) ou distâncias (metros para pés, quilômetros para milhas).
    *   **Critérios de Sucesso:** Interface intuitiva com campos de entrada para o valor a ser convertido e botões para selecionar o tipo de conversão. O resultado deve ser exibido em um campo de texto atualizado dinamicamente.
    *   **Verificação:** Compare os resultados do seu aplicativo com conversores online para garantir a precisão. Este desafio reforça o uso de `EditText`, `TextView`, `Button` e manipulação de eventos.

2.  **Lista de Tarefas Simples:** Crie um app para gerenciar uma lista de tarefas. O usuário deve poder adicionar novas tarefas, marcar como concluídas e excluir tarefas existentes.
    *   **Critérios de Sucesso:** Utilizar um `RecyclerView` para exibir a lista de tarefas. Cada item da lista deve ter um checkbox para marcar como concluído e um botão para excluir. Os dados devem ser persistidos localmente (e.g., usando `SharedPreferences` ou um banco de dados SQLite).
    *   **Verificação:** Certifique-se de que as tarefas são salvas corretamente e permanecem na lista mesmo após o app ser reiniciado. Este desafio pratica o uso de listas, persistência de dados e manipulação de UI.

3.  **Calculadora de IMC:** Implemente uma calculadora de Índice de Massa Corporal (IMC). O aplicativo deve receber a altura e o peso do usuário, calcular o IMC e exibir o resultado juntamente com uma categoria de peso (e.g., abaixo do peso, normal, sobrepeso, obesidade).
    *   **Critérios de Sucesso:** Validar as entradas do usuário para garantir que altura e peso sejam valores numéricos válidos. Exibir uma mensagem de erro caso as entradas sejam inválidas. A categoria de peso deve ser exibida de forma clara e concisa.
    *   **Verificação:** Use uma calculadora de IMC online para comparar os resultados e garantir a precisão. Este desafio trabalha com entrada de dados, cálculos e lógica condicional.

**Intermediário:**

4.  **Aplicativo de Previsão do Tempo:** Integre uma API de previsão do tempo (como a OpenWeatherMap) para exibir a previsão atual e dos próximos dias para uma determinada cidade.
    *   **Critérios de Sucesso:** O usuário deve poder inserir o nome de uma cidade e receber a previsão do tempo (temperatura, umidade, descrição do clima, etc.). A interface deve exibir informações relevantes de forma organizada e fácil de entender. Trate erros de rede e exiba mensagens amigáveis caso a API não esteja disponível.
    *   **Verificação:** Compare a previsão do seu aplicativo com outros serviços de previsão do tempo para garantir a precisão. Este desafio envolve requisições HTTP, parsing de JSON e manipulação de dados assíncronos.

5.  **Gerenciador de Notas com Banco de Dados:** Desenvolva um aplicativo para criar, editar e excluir notas. Utilize um banco de dados SQLite para armazenar as notas.
    *   **Critérios de Sucesso:** O aplicativo deve permitir criar notas com título e conteúdo. As notas devem ser listadas em um `RecyclerView` e o usuário deve poder tocar em uma nota para editar ou excluir. Utilize um `ContentProvider` para acessar os dados do banco de dados de forma segura e eficiente.
    *   **Verificação:** Crie várias notas, edite-as e exclua-as para garantir que o aplicativo está funcionando corretamente. Este desafio reforça o uso de bancos de dados, `RecyclerView`, `ContentProvider` e manipulação de dados.

6.  **Reprodutor de Áudio Simples:** Crie um aplicativo que reproduza arquivos de áudio armazenados no dispositivo. O aplicativo deve ter controles para reproduzir, pausar, avançar, retroceder e exibir o tempo atual da reprodução.
    *   **Critérios de Sucesso:** O aplicativo deve ser capaz de listar os arquivos de áudio no dispositivo e permitir que o usuário selecione um arquivo para reproduzir. Utilize um `Service` para reproduzir o áudio em segundo plano, mesmo quando o aplicativo não está em primeiro plano.
    *   **Verificação:** Teste o aplicativo com diferentes formatos de áudio e verifique se a reprodução funciona corretamente em segundo plano. Este desafio trabalha com manipulação de arquivos, reprodução de áudio e `Service`.

**Avançado:**

7.  **Aplicativo de Realidade Aumentada:** Utilize o ARCore para criar um aplicativo de realidade aumentada que permita ao usuário colocar objetos virtuais em seu ambiente.
    *   **Critérios de Sucesso:** O aplicativo deve detectar planos no ambiente e permitir que o usuário toque na tela para colocar objetos virtuais (e.g., modelos 3D simples). Os objetos devem permanecer fixos no ambiente mesmo quando o usuário move o dispositivo.
    *   **Verificação:** Teste o aplicativo em diferentes ambientes e verifique se a detecção de planos e a colocação de objetos virtuais funcionam corretamente. Este desafio envolve o uso de ARCore, OpenGL e conceitos de realidade aumentada.

8.  **Chat em Tempo Real com Firebase:** Implemente um aplicativo de chat em tempo real utilizando o Firebase Realtime Database e o Firebase Authentication.
    *   **Critérios de Sucesso:** O usuário deve poder criar uma conta e fazer login no aplicativo. As mensagens devem ser exibidas em tempo real para todos os usuários conectados. Utilize o Firebase Cloud Messaging para enviar notificações push quando novas mensagens são recebidas.
    *   **Verificação:** Teste o aplicativo com múltiplos usuários e verifique se as mensagens são sincronizadas corretamente em tempo real. Este desafio reforça o uso de Firebase, autenticação, bancos de dados em tempo real e notificações push.

9.  **Reconhecimento de Imagem com Machine Learning:** Integre o TensorFlow Lite para criar um aplicativo que reconheça objetos em imagens capturadas pela câmera do dispositivo.
    *   **Critérios de Sucesso:** O aplicativo deve exibir a imagem capturada pela câmera e identificar os objetos presentes na imagem com uma certa precisão. Utilize um modelo de machine learning pré-treinado (e.g., MobileNet) para realizar o reconhecimento de imagem.
    *   **Verificação:** Teste o aplicativo com diferentes objetos e verifique se o reconhecimento de imagem funciona corretamente. Este desafio envolve o uso de TensorFlow Lite, processamento de imagem e conceitos de machine learning.

